In [3]:
import pandas as pd
df = pd.read_csv("data\preb_churn_data.csv")

In [4]:
df.head()

,Customerid,Gender,Senior_citizen,Is_married,Dependents,Tenure,Phone_service,Dual,Internet_service,Online_security,...,Device_protection,Tech_support,Streaming_tv,Streaming_movies,Contract,Paperless_billing,Payment_method,Monthly_charges,Total_charges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [27]:
df['Total_charges'].value_counts().index

Float64Index([   20.2,   19.75,   20.05,    19.9,   19.65,   19.55,    45.3,
                19.45,   20.25,   20.15,
              ...
              3306.85,  424.75, 6565.85,  2117.2,  203.95,  6849.4,  692.35,
               130.15,  3211.9,  6844.5],
             dtype='float64', length=6530)

In [29]:
from langchain.prompts import PromptTemplate
from src.model import  get_model


WARNING! add_to_git_credential is not default parameter.
                    add_to_git_credential was transferred to model_kwargs.
                    Please make sure that add_to_git_credential is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\oscar\.cache\huggingface\token
Login successful


In [54]:
input_prompt ='''
**Task:** Extract and return customer information in a specific order as a list.
only return list not add any information or notes or anything.


**Instructions:**
1. **Extract the following features from the provided query in the exact order listed below.**
2. **For each feature, select the value from the given options only.**
3. **Return the values as a Python list in the specified order. Ensure the values are case-sensitive and match exactly as described in the options.**

**Order of Features and Allowed Values:**
1. **Gender**: 'Male' or 'Female'
2. **Senior_citizen**: 1 (for Yes) or 0 (for No)
3. **Is_married**: 'Yes' or 'No'
4. **Dependents**: 'Yes' or 'No'
5. **Tenure**: A number representing the number of months
6. **Phone_service**: 'Yes' or 'No'
7. **Dual**: 'Yes' or 'No'
8. **Internet_service**: 'DSL', 'Fiber optic', or 'No'
9. **Online_security**: 'Yes', 'No', or 'No internet service'
10. **Online_backup**: 'Yes', 'No', or 'No internet service'
11. **Device_protection**: 'Yes', 'No', or 'No internet service'
12. **Tech_support**: 'Yes', 'No', or 'No internet service'
13. **Streaming_tv**: 'Yes', 'No', or 'No internet service'
14. **Streaming_movies**: 'Yes', 'No', or 'No internet service'
15. **Contract**: 'Month-to-month', 'One year', or 'Two year'
16. **Paperless_billing**: 'Yes' or 'No'
17. **Payment_method**: 'Electronic check', 'Mailed check', 'Bank transfer (automatic)', or 'Credit card (automatic)'
18. **Monthly_charges**: A number representing the monthly charges
19. **Total_charges**: A number representing the total charges

Important:

-The list must strictly follow the order mentioned.
-All categorical values must match the provided options exactly.
-Do not return any additional text or explanation, only the list.

customer information :

{content}

'''


In [55]:
prompt = PromptTemplate(template=input_prompt, input_variables=['content'])

In [63]:
chain = prompt | get_model() | eval

In [64]:
test = '''Jane Smith is a Female customer who is 70 years old and is a senior citizen. She has been with the company for 12 months. Jane is married and has two children. She does not have a phone service but has a dual-line connection. Her internet service provider is DSL. She has online security enabled but does not have online backup or device protection. She does receive technical support and subscribes to streaming TV but not to streaming movies. Jane has a two-year contract and prefers receiving paper bills. She pays through a mailed check, with monthly charges of 60.2, and her total charges so far are 722.4.
'''

In [65]:
res = chain.invoke({"content":test})

In [66]:
res

['Female',
 0,
 'Yes',
 'Yes',
 12,
 'No',
 'Yes',
 'DSL',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'Two year',
 'Yes',
 'Mailed check',
 60.2,
 722.4]